In [4]:
# Set conda env to dino4cells
data_dir = '/mnt/vast/hpc/LDB/LDB_data/microscopy/HPA/whole_images/'
import os
import torch

One of the many issues: train_IDs are massively duplicated (900,000) in train_ids.pth, but Wolf has cleaned it already:


"So, the files we should use in general are under /mnt/vast/hpc/LDB/LDB_data/microscopy/HPA/whole_images/:
Master_fovHPA_512.csv (all HPA images that we should use)
fovHPA_train_IDs_proteinloc.pth (cleaned and de-duplicated train_IDs)
fovHPA_valid_IDs_proteinloc.pth (cleaned and de-duplicated valid_IDs) " -Wolf

In [5]:

os.listdir(data_dir)

['.ipynb_checkpoints',
 '03-02-2024_4d_train_umap_embeddings',
 '512_whole_images',
 '512_whole_images_kaggle_test',
 'DINO_features_HPA_FOV_test.pth',
 'DINO_features_HPA_FOV_train.pth',
 'DINO_features_HPA_FOV_valid.pth',
 'DINO_features_for_HPA_FOV.pth',
 'Kaggle_test_fovHPA_512.csv',
 'Master_fovHPA_512.csv',
 'Master_fovHPA_512_1d.csv',
 'Master_fovHPA_512_save.csv',
 'fovHPA_1d_512.csv',
 'fovHPA_train_IDs_proteinloc.pth',
 'fovHPA_valid_IDs_proteinloc.pth',
 'kaggle_test_data_direct.csv',
 'large_whole_images',
 'sample_submission.csv',
 'submission_whole_images_4d_2_19.csv',
 'submission_whole_images_4d_3epochs_03_02.csv',
 'train',
 'train.csv',
 'zenodo']

In [10]:
train_ids = torch.load(os.path.join(data_dir, 'fovHPA_train_IDs_proteinloc.pth'))
valid_ids = torch.load(os.path.join(data_dir, 'fovHPA_valid_IDs_proteinloc.pth'))

In [11]:
len(valid_ids)

18028

In [12]:
import pandas as pd
fov_img = pd.read_csv(os.path.join(data_dir, 'Master_fovHPA_512.csv'))

/tmp/ipykernel_30269/1435998313.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [17]:

# Create a new column with the full path
fov_img['file'] = fov_img['Path'].apply(lambda x: f"/mnt/vast/hpc/LDB/LDB_data/microscopy/HPA/{x}")


In [22]:
fov_img.to_csv(os.path.join(data_dir, 'Master_fovHPA_512_with_file.csv'))

Known issue: first inconsistency: len of train and valid in that dir is 113545 total, len of train_ids in zenodo is 117883. 

Wolf's comment: its due to duplication. 

### YAML Creation

Keeping the config of the broad_institute repo, removing duplicates and classification section, exploratory_configs/2024_05_27_config_HPA_FOV_rerun.yaml. changed it to vit_base. making submission script: exploratory_configs/2024_05_27_run_end_to_end.sh 

In [ ]:
sbatch --partition GEN-GPU --gres=gpu:1 -t 48:00:00 exploratory_configs/2024_05_27_run_end_to_end.sh

CUDA_VISIBLE_DEVICES=0 python /mnt/vast/hpc/homes/ndv2107/Dino4Cells_analysis/run_dino.py --config /mnt/vast/hpc/homes/ndv2107/Dino4Cells_analysis/exploratory_configs/2024_05_27_config_HPA_FOV_rerun.yaml 2>&1 | tee log.txt

In [ ]:
Error: ModuleNotFoundError: No module named 'kornia'
Trying a fix: (dino4cells) ndv2107@hgrclogin01:~$ /home/ndv2107/miniconda3/envs/dino4cells/bin/pip install kornia
Error: DINO: error: unrecognized arguments: --local-rank=0
Trying a fix: replace run_dino.py with run_end_to_end.py
Error: 
  import pandas as pd
Namespace(config='/mnt/vast/hpc/homes/ndv2107/Dino4Cells_analysis/exploratory_configs/2024_05_27_config_HPA_FOV_rerun.yaml', checkpoint_frq=50, checksize=3, check_batch=None, test_last=False, train='True', test='True', dist_url='env://')
[]
Traceback (most recent call last):
  File "/mnt/vast/hpc/homes/ndv2107/Dino4Cells_analysis/run_end_to_end.py", line 658, in <module>
    if "--local_rank" in unknown[0]:
                         ~~~~~~~^^^
IndexError: list index out of range

What is this local_rank param it's complaining about? Not in the yaml. Root Cause (first observed failure):
[0]:
  time      : 2024-05-27_14:24:38
  host      : gpunode01
  rank      : 0 (local_rank: 0)
  exitcode  : 2 (pid: 2519950)
  error_file: <N/A>
  traceback : To enable traceback see: https://pytorch.org/docs/stable/elastic/errors.html
============================================================
CUDA_VISIBLE_DEVICES=. python3 -m torch.distributed.launch --master_port=29501 --nproc_per_node=1 main_dino.py --arch vit_base --output_dir output/ --data_path /mnt/vast/hpc/LDB/LDB_data/microscopy/HPA/whole_images/Master_fovHPA_512.csv  --saveckp_freq 50 --batch_size_per_gpu 24 --num_channels 5 --patch_size 16 --local_crops_scale 0.2 0.5 --epochs 10 --config /mnt/vast/hpc/homes/ndv2107/Dino4Cells_analysis/exploratory_configs/2024_05_27_config_HPA_FOV_rerun.yaml --center_momentum 0.9 --lr 0.0005

```RuntimeError: ProcessGroupNCCL is only supported with GPUs, no GPUs found!````

In [ ]:
Trying this:
sbatch --partition GEN-GPU --gres=gpu:0 -t 48:00:00 exploratory_configs/2024_05_27_run_end_to_end.sh
Submitted batch job 318480
(dino4cells) ndv2107@hgrclogin01:~/Dino4Cells_analysis$ tail -f slurm-318480.out

the fix is to run `sbatch --partition GEN-GPU --gres=gpu:1 -t 48:00:00 exploratory_configs/2024_05_27_run_end_to_end.sh`
on this git commit:

https://github.com/broadinstitute/Dino4Cells_analysis/commit/f828b138ae69d6e27d04f173c7f235f0adba0706